Download data from FastF1

In [1]:
import fastf1
from scripts import ds_ultils,data_cleaning
from scripts.f1_ultils import convert_time_miliseconds,find_fastest_lap, fill_mean_parameter,clean_q3_times

from scripts.fastf1_data_download import f1_telemetry_data , combine_telemetry
import numpy as np
import pandas as pd
    


In [2]:
random = np.random.RandomState(1234).randint(20,size=3)
random

array([15, 19,  6])

In [2]:
ergast_combined_df=pd.read_csv('./data/clean/combined_ergast_clean.csv')

C:\Users\ben44\AppData\Local\Temp\ipykernel_24700\450351449.py:1: DtypeWarning: Columns (8,9,39,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  ergast_combined_df=pd.read_csv('./data/clean/combined_ergast_clean.csv')


In [5]:
circuits=list(ergast_combined_df.loc[ergast_combined_df['year']==2018,'name'].unique())


In [6]:
circuits=list(ergast_combined_df.loc[ergast_combined_df['year']==2018,'name'].unique())
circuits_2018=[]
for i in random:
    if circuits[i] not in (['Monaco Grand Prix','Spanish Grand Prix']):
        circuits_2018.append(circuits[i])
    else:
        circuits_2018.append(circuits[i+2])
circuits_2018.append('Monaco Grand Prix')
circuits_2018.append('Spanish Grand Prix')
circuits_2018

['Austrian Grand Prix',
 'Russian Grand Prix',
 'Hungarian Grand Prix',
 'Monaco Grand Prix',
 'Spanish Grand Prix']

In [ ]:
event_data={}
for circuit in circuits_2018:
    event_data[circuit] =f1_telemetry_data(2018,circuit)


In [ ]:
event_data.keys()

dict_keys(['Austrian Grand Prix', 'Russian Grand Prix', 'Hungarian Grand Prix', 'Monaco Grand Prix', 'Spanish Grand Prix'])

In [ ]:
combined_2018_data = combine_telemetry(event_data,ergast_combined_df,session='qualifying',year=2018)

In [ ]:
combined_2018_data.to_pickle('./data/clean/2018_telemetry_subsetcircuits_data.pkl',compression='gzip')

In [ ]:
len(combined_2018_data)

549163

In [ ]:
combined_2018_data['circuitRef'].unique()

array(['red_bull_ring', 'sochi', 'hungaroring', 'monaco', 'catalunya'],
      dtype=object)

In [ ]:
event_data={}
for circuit in circuits_2018:
    event_data[circuit] =f1_telemetry_data(2019,circuit)


In [ ]:
combined_2019_data = combine_telemetry(event_data,ergast_combined_df,session='qualifying',year=2019)

In [ ]:
combined_2019_data.to_pickle('./data/clean/2019_telemetry_subsetcircuits_data.pkl',compression='gzip')

In [ ]:
combined_2019_data.head()

,raceId,year,circuitId,name,date,time,quali_date,quali_time,sprint_date,sprint_time,lat_x,lng_x,race_temp,race_precip,race_humidity,quali_temp,quali_precip,quali_humidity,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,race_condition_wiki,quali_condition_clean,circuitRef,location,country,alt,qualifyId,driverId,constructorId,quali_position,q1,q2,q3,resultId_x,grid_sprint,positionText_sprint,...,qualifying_SpeedI1,qualifying_SpeedI2,qualifying_SpeedFL,qualifying_SpeedST,qualifying_IsPersonalBest,qualifying_Compound,qualifying_TyreLife,qualifying_FreshTyre,qualifying_Stint,qualifying_LapStartTime,qualifying_Team,qualifying_Driver,qualifying_TrackStatus,qualifying_IsAccurate,qualifying_LapStartDate,qualifying_AirTemp,qualifying_Humidity,qualifying_Pressure,qualifying_Rainfall,qualifying_TrackTemp,qualifying_WindDirection,qualifying_WindSpeed,qualifying_Date,qualifying_SessionTime,qualifying_DriverAhead,qualifying_DistanceToDriverAhead,qualifying_Time_y,qualifying_RPM,qualifying_Speed,qualifying_nGear,qualifying_Throttle,qualifying_Brake,qualifying_DRS,qualifying_Source,qualifying_Distance,qualifying_RelativeDistance,qualifying_Status,qualifying_X,qualifying_Y,qualifying_Z
0,1018,2019,70,Austrian Grand Prix,2019-06-30,13:10:00,NaN,NaN,NaN,NaN,47.2197,14.7647,32.733333,0.0,25.193333,27.7,0.0,32.366667,NaN,NaN,NaN,NaN,NaN,NaN,dry,dry,red_bull_ring,Spielberg,Austria,678.0,8118,844,6,1,1:04.138,1:03.378,1:03.003,NaN,NaN,NaN,...,329.0,249.0,297.0,325.0,False,MEDIUM,2.0,True,1.0,0 days 00:20:54.451000,Ferrari,LEC,1,True,2019-06-29 13:05:56.707,29.0,28.1,942.0,False,51.9,76,2.3,2019-06-29 13:05:56.707,0 days 00:20:54.451000,,124.540278,0 days 00:00:00,10855.0,303.0,8.0,100.0,False,12.0,interpolation,0.140054,0.000032,OnTrack,1447.0,-1090.0,7215.0
1,1018,2019,70,Austrian Grand Prix,2019-06-30,13:10:00,NaN,NaN,NaN,NaN,47.2197,14.7647,32.733333,0.0,25.193333,27.7,0.0,32.366667,NaN,NaN,NaN,NaN,NaN,NaN,dry,dry,red_bull_ring,Spielberg,Austria,678.0,8118,844,6,1,1:04.138,1:03.378,1:03.003,NaN,NaN,NaN,...,329.0,249.0,297.0,325.0,False,MEDIUM,2.0,True,1.0,0 days 00:20:54.451000,Ferrari,LEC,1,True,2019-06-29 13:05:56.707,29.0,28.1,942.0,False,51.9,76,2.3,2019-06-29 13:05:56.809,0 days 00:20:54.553000,,124.540278,0 days 00:00:00.102000,10878.0,304.0,8.0,100.0,False,12.0,car,8.766667,0.002019,OnTrack,1374.0,-1109.0,7215.0
2,1018,2019,70,Austrian Grand Prix,2019-06-30,13:10:00,NaN,NaN,NaN,NaN,47.2197,14.7647,32.733333,0.0,25.193333,27.7,0.0,32.366667,NaN,NaN,NaN,NaN,NaN,NaN,dry,dry,red_bull_ring,Spielberg,Austria,678.0,8118,844,6,1,1:04.138,1:03.378,1:03.003,NaN,NaN,NaN,...,329.0,249.0,297.0,325.0,False,MEDIUM,2.0,True,1.0,0 days 00:20:54.451000,Ferrari,LEC,1,True,2019-06-29 13:05:56.707,29.0,28.1,942.0,False,51.9,76,2.3,2019-06-29 13:05:56.952,0 days 00:20:54.696000,,124.540278,0 days 00:00:00.245000,10956.0,305.0,8.0,100.0,False,12.0,pos,20.895643,0.004812,OnTrack,1272.0,-1137.0,7215.0
3,1018,2019,70,Austrian Grand Prix,2019-06-30,13:10:00,NaN,NaN,NaN,NaN,47.2197,14.7647,32.733333,0.0,25.193333,27.7,0.0,32.366667,NaN,NaN,NaN,NaN,NaN,NaN,dry,dry,red_bull_ring,Spielberg,Austria,678.0,8118,844,6,1,1:04.138,1:03.378,1:03.003,NaN,NaN,NaN,...,329.0,249.0,297.0,325.0,False,MEDIUM,2.0,True,1.0,0 days 00:20:54.451000,Ferrari,LEC,1,True,2019-06-29 13:05:56.707,29.0,28.1,942.0,False,51.9,76,2.3,2019-06-29 13:05:57.048,0 days 00:20:54.792000,44,124.540278,0 days 00:00:00.341000,11035.0,306.0,8.0,100.0,False,12.0,car,29.081667,0.006697,OnTrack,1204.0,-1155.0,7214.0
4,1018,2019,70,Austrian Grand Prix,2019-06-30,13:10:00,NaN,NaN,NaN,NaN,47.2197,14.7647,32.733333,0.0,25.193333,27.7,0.0,32.366667,NaN,NaN,NaN,NaN,NaN,NaN,dry,dry,red_bull_ring,Spielberg,Austria,678.0,8118,844,6,1,1:04.138,1:03.378,1:03.003,NaN,NaN,NaN,...,329.0,249.0,297.0,325.0,False,MEDIUM,2.0,True,1.0,0 days 00:20:54.451000,Ferrari,LEC,1,True,2019-06-29 13:05:56.707,29.0,28.1,942.0,False,51.9,76,2.3,2019-06-29 13:05:57.252,0 days 00:20:54.996000,44,124.656944,0 days 00:00:00.545000,11068.0,30

In [ ]:
combined_telemetry_data=pd.concat([combined_2018_data,combined_2019_data])

In [ ]:
combined_telemetry_data.to_pickle('./data/clean/years_18_19_telemetry_subsetcircuits_data.pkl',compression='gzip')

In [2]:
combined_telemetry_data=pd.read_pickle('./data/clean/years_18_19_telemetry_subsetcircuits_data.pkl',compression='gzip')

C:\Users\ben44\AppData\Local\Temp\ipykernel_13328\3422113839.py:1: DtypeWarning: Columns (35,36,49,90,93,99,104,117) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_telemetry_data=pd.read_csv('./data/clean/years_18_19_telemetry_subsetcircuits_data.csv')


In [3]:
ds_ultils.nan_checker(combined_telemetry_data)

This dataframe has 21719605 NaN values
 The NaN values come from: ['quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'q1', 'q2', 'q3', 'resultId_x', 'grid_sprint', 'positionText_sprint', 'positionOrder_sprint', 'points_sprint', 'laps_sprint', 'milliseconds_sprint', 'fastestLap_sprint', 'fastestLapTime_sprint', 'status_sprint', 'milliseconds', 'fastestLap', 'fastestLapTime', 'fastestLapSpeed', 'qualifying_lapId', 'qualifying_Time_x', 'qualifying_DriverNumber', 'qualifying_LapTime', 'qualifying_LapNumber', 'qualifying_Sector1Time', 'qualifying_Sector2Time', 'qualifying_Sector3Time', 'qualifying_Sector1SessionTime', 'qualifying_Sector2SessionTime', 'qualifying_Sector3SessionTime', 'qualifying_SpeedI1', 'qualifying_SpeedI2', 'qualifying_SpeedFL', 'qualifying_SpeedST', 'qualifying_IsPersonalBest', 'qualifying_Compound', 'qualifying_TyreLife', 'qualifying_FreshTyre', 'qualifying_Stint', 'qualifying_LapStartTime',

In [4]:
combined_telemetry_data['qualifying_LapStartDate']

0          2018-06-30 13:03:47.133
1          2018-06-30 13:03:47.133
2          2018-06-30 13:03:47.133
3          2018-06-30 13:03:47.133
4          2018-06-30 13:03:47.133
                    ...           
1018653    2019-05-11 13:16:36.564
1018654    2019-05-11 13:16:36.564
1018655    2019-05-11 13:16:36.564
1018656    2019-05-11 13:16:36.564
1018657    2019-05-11 13:16:36.564
Name: qualifying_LapStartDate, Length: 1018658, dtype: object

For 2018 and 2019 there was no sprint races,so we can drop those columns. In addittion the date and time of the qualifying can be dropped since we have the same data replicated per lap in the telemetry data.


In [5]:
qualifying_data_clean = combined_telemetry_data.copy()

In [6]:
qualifying_data_clean.drop(columns=['quali_date', 'quali_time', 'sprint_date',
 'sprint_time', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 
 'fp3_time', 'grid_sprint', 'positionText_sprint', 'positionOrder_sprint', 
 'points_sprint', 'laps_sprint', 'milliseconds_sprint', 'fastestLap_sprint', 
 'fastestLapTime_sprint', 'status_sprint'],axis=1,inplace=True)

In [7]:
(qualifying_data_clean['resultId_x'].isna()).sum()/len(qualifying_data_clean)
#resultID is for the sprint races with no results so it is all nan's - we can drop columns

1.0

In [8]:
qualifying_data_clean.drop(columns='resultId_x',axis=1,inplace=True)

We are focusing on predicting qualifying performance for specific, drivers, cars and circuits. Therefore we can remove data on the race for this analysis.

In [9]:
qualifying_data_clean['milliseconds'].isna().sum()/len(qualifying_data_clean)

0.5199615572645578

In [10]:
qualifying_data_clean.drop(columns=['laps',
 'milliseconds',
 'fastestLap',
 'rank',
 'fastestLapTime',
 'fastestLapSpeed',
 'status'],axis=1,inplace=True)

Since we have detailed weather conditions information for each lap we can remove weather condition information we collected previously as this is a duplicate

In [11]:
qualifying_data_clean.drop(columns=['race_temp',
 'race_precip',
 'race_humidity',
 'quali_temp',
 'quali_precip',
 'quali_humidity',
 'race_condition_wiki',
 'quali_condition_clean'],axis=1,inplace=True)

In [12]:
print('The number of unqiue circuit ref matches the circuitId :',len(qualifying_data_clean.circuitId.unique()) == len(qualifying_data_clean.circuitRef.unique()))
print('The number of unqiue Driver ref matches the driverId :',len(qualifying_data_clean.driverId.unique()) == len(qualifying_data_clean.driverRef.unique()))
print('The number of unqiue constructor ref matches the constructorId :',len(qualifying_data_clean.constructorId.unique()) == len(qualifying_data_clean.constructorRef.unique()))


The number of unqiue circuit ref matches the circuitId : True
The number of unqiue Driver ref matches the driverId : True
The number of unqiue constructor ref matches the constructorId : True


In [13]:
qualifying_data_clean.drop(columns=['circuitId','driverId','constructorId'],axis=1,inplace=True)

The other id's for result, driver standinds constructor standings can be now dropped as we have successfully merged the data and kept unique entries for driver, car and circuit combinations

In [14]:
qualifying_data_clean.drop(columns=['resultId_y',
'driverStandingsId',
'constructorStandingsId',
'constructorResultsId'],axis=1,inplace=True)

## Now to merge Q1,2,3 times to retain the fastest time and convert to milliseconds

In [15]:
((qualifying_data_clean['q1'].isna()).sum()/len(qualifying_data_clean))*100

0.00019633674893830905

In [17]:
qualifying_data_clean[qualifying_data_clean['q1'].isna()]

,raceId,year,name,date,time,lat_x,lng_x,circuitRef,location,country,...,qualifying_Throttle,qualifying_Brake,qualifying_DRS,qualifying_Source,qualifying_Distance,qualifying_RelativeDistance,qualifying_Status,qualifying_X,qualifying_Y,qualifying_Z
426623,994,2018,Monaco Grand Prix,2018-05-27,13:10:00,43.7347,7.42056,monaco,Monte-Carlo,Monaco,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
549162,993,2018,Spanish Grand Prix,2018-05-13,13:10:00,41.5700,2.26111,catalunya,Montmeló,Spain,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
qualifying_data_clean = qualifying_data_clean[qualifying_data_clean.index.isin(qualifying_data_clean['q1'].dropna().index)]

In [19]:
#make np.NaN floats strings for conversion to milliseconds
qualifying_data_clean['q2'].fillna('nan',inplace=True)
qualifying_data_clean['q3'].fillna('nan',inplace=True)

In [20]:
qualifying_data_clean['q1_milliseconds']=qualifying_data_clean['q1'].apply(convert_time_miliseconds)

In [21]:
qualifying_data_clean['q2_milliseconds']=qualifying_data_clean['q2'].apply(convert_time_miliseconds)
qualifying_data_clean['q3_milliseconds']=qualifying_data_clean['q3'].apply(convert_time_miliseconds)

In [22]:
qualifying_data_clean=clean_q3_times(qualifying_data_clean,'q1_milliseconds','q2_milliseconds','q3_milliseconds','fastest_lap_milliseconds')

In [23]:
#validate that worked as expected
qualifying_data_clean[['fastest_lap_milliseconds','q1_milliseconds','q2_milliseconds','q3_milliseconds','q1','q2','q3']]

,fastest_lap_milliseconds,q1_milliseconds,q2_milliseconds,q3_milliseconds,q1,q2,q3
0,63130.0,64175.0,63756.0,63130.0,1:04.175,1:03.756,1:03.130
1,63130.0,64175.0,63756.0,63130.0,1:04.175,1:03.756,1:03.130
2,63130.0,64175.0,63756.0,63130.0,1:04.175,1:03.756,1:03.130
3,63130.0,64175.0,63756.0,63130.0,1:04.175,1:03.756,1:03.130
4,63130.0,64175.0,63756.0,63130.0,1:04.175,1:03.756,1:03.130
...,...,...,...,...,...,...,...
1018653,80254.0,80254.0,NaN,NaN,1:20.254,nan,nan
1018654,80254.0,80254.0,NaN,NaN,1:20.254,nan,nan
1018655,80254.0,80254.0,NaN,NaN,1:20.254,nan,nan
1018656,80254.0,80254.0,NaN,NaN,1:20.254,nan,nan


In [24]:
(qualifying_data_clean['fastest_lap_milliseconds'].isna()).sum()

0

Now we can drop the intermediate columns and the individual session fastest times and retain just the fastest time for that circuit

In [25]:
qualifying_data_clean.drop(columns=['q1_milliseconds',
'q2_milliseconds',
'q3_milliseconds',
'q1',
'q2',
'q3'
], axis=1,inplace=True)

Date is not the date of qualifying but of the race, we can replace that with qualifying lap data.
time is also not of qualifying but of the race so can be dropped
Location is a duplicate of Lat, Lng and Alt,
grid is quali_position but accounts for penalties. This is not linked to pure performance ( such as engine penalties for replacing engines etc. Therefore we will drop grid)

number should be a repeat of qualifying driver number

In [26]:
qualifying_data_clean['date']=qualifying_data_clean['qualifying_Date'].str.split(' ').str[0]

In [27]:
(qualifying_data_clean['number'] != qualifying_data_clean['qualifying_DriverNumber']).sum()

1

In [28]:
qualifying_data_clean.loc[qualifying_data_clean['number'] != qualifying_data_clean['qualifying_DriverNumber'],['number','qualifying_DriverNumber','circuitRef','qualifying_lapId']]

,number,qualifying_DriverNumber,circuitRef,qualifying_lapId
717562,26.0,NaN,sochi,NaN


Missing  lap data for car 26 in Sochi. Will drop this row of data

In [29]:
qualifying_data_clean.drop(index=717562,axis=0,inplace=True)

In [30]:
(qualifying_data_clean['number'] != qualifying_data_clean['qualifying_DriverNumber']).sum()

0

In [31]:
qualifying_data_clean.drop(columns=['number','location','grid'],axis=1,inplace=True)

PositionText and positionOrder are duplicates of race result, will combine

In [32]:
qualifying_data_clean['positionOrder'].unique()

array([19., 16.,  3.,  2.,  1.,  4., 18.,  5., 12., 20.,  6., 11.,  9.,
        8., 14., 15.,  7., 13., 17., 10.])

In [33]:
qualifying_data_clean['positionText'].unique()

array(['R', '3', '2', 2, 1, 4, '4', '5', '12', '6', 6, 11, 9, 8, 14, 15,
       7, '7', '13', '10', 3, 10, 13, 5, '17', '14', '18', '16', '15',
       '11', 12, 16, 17, '8', '9', 19, 18, 20, '1', '19'], dtype=object)

In [34]:
qualifying_data_clean[['positionText','positionOrder']].iloc[1000:1010]

,positionText,positionOrder
1000,R,19.0
1001,R,19.0
1002,R,19.0
1003,R,19.0
1004,R,19.0
1005,R,19.0
1006,R,19.0
1007,R,19.0
1008,R,19.0
1009,R,19.0


Position Text accounts for retired, position Order is just a float which assigns cars a result even if they don't finish. The first car to retire is assigned last position and the next car to retire next to last etc.. Since we are focusing on qualifying car performance we will drop positionText

In [35]:
qualifying_data_clean.drop(columns=['positionText'],axis=1,inplace=True)

We will convert dob into age for this analysis to see if age has an impact on pole positions

In [36]:
qualifying_data_clean['dob']=qualifying_data_clean['dob'].astype('datetime64')
qualifying_data_clean['date']=qualifying_data_clean['date'].astype('datetime64')

In [37]:
# inspired from stackoverflow https://stackoverflow.com/questions/26788854/pandas-get-the-age-from-a-date-example-date-of-birth
qualifying_data_clean['age']=(qualifying_data_clean['date'] - qualifying_data_clean['dob']).astype('<m8[Y]')

In [38]:
qualifying_data_clean.drop(columns=['dob'],axis=1,inplace=True)

In [39]:
ds_ultils.nan_checker(qualifying_data_clean)


This dataframe has 178042 NaN values
 The NaN values come from: ['qualifying_LapTime', 'qualifying_SpeedI1', 'qualifying_SpeedI2', 'qualifying_SpeedFL', 'qualifying_SpeedST', 'qualifying_Compound', 'qualifying_TyreLife', 'qualifying_FreshTyre', 'qualifying_Stint', 'qualifying_DriverAhead', 'qualifying_DistanceToDriverAhead']. 



In [40]:
qualifying_data_clean.loc[qualifying_data_clean['qualifying_LapTime'].isna(),['qualifying_LapTime', 'qualifying_SpeedI1', 'qualifying_SpeedI2', 'qualifying_SpeedFL', 'qualifying_SpeedST', 'qualifying_Compound', 'qualifying_TyreLife', 'qualifying_FreshTyre', 'qualifying_Stint', 'qualifying_DriverAhead', 'qualifying_DistanceToDriverAhead','qualifying_DriverNumber']]

,qualifying_LapTime,qualifying_SpeedI1,qualifying_SpeedI2,qualifying_SpeedFL,qualifying_SpeedST,qualifying_Compound,qualifying_TyreLife,qualifying_FreshTyre,qualifying_Stint,qualifying_DriverAhead,qualifying_DistanceToDriverAhead,qualifying_DriverNumber
95414,NaN,108.0,253.0,177.0,140.0,HYPERSOFT,5.0,True,1.0,NaN,1242.801111,44.0
95415,NaN,108.0,253.0,177.0,140.0,HYPERSOFT,5.0,True,1.0,NaN,1242.801111,44.0
95416,NaN,108.0,253.0,177.0,140.0,HYPERSOFT,5.0,True,1.0,NaN,1242.801111,44.0
95417,NaN,108.0,253.0,177.0,140.0,HYPERSOFT,5.0,True,1.0,NaN,1242.801111,44.0
95418,NaN,108.0,253.0,177.0,140.0,HYPERSOFT,5.0,True,1.0,77.0,1242.801111,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...
677167,NaN,117.0,255.0,172.0,244.0,SOFT,3.0,True,1.0,23.0,322.196944,8.0
677168,NaN,117.0,255.0,172.0,244.0,SOFT,3.0,True,1.0,23.0,324.830278,8.0
677169,NaN,117.0,255.0,172.0,244.0,SOFT,3.0,True,1.0,23.0,327.054306,8.0
677170,NaN,117.0,255.0,172.0,244.0,SOFT,3.0,True,1.0,23.0,329.278333,8.0


To measure pure car performance we do not need to know which car is ahead. Therefore we can remove driver ahead

In [41]:
qualifying_data_clean.drop(columns=['qualifying_DriverAhead'],axis=1,inplace=True)

In [45]:
(qualifying_data_clean['qualifying_LapTime'].isna().sum()/len(qualifying_data_clean))*100

1.0603197353372829

IsAccurate (bool): Indicates that the lap start and end time are synced correctly with other laps. Do not confuse this with the accuracy of the lap time or sector times. They are always considered to be accurate if they exist! If this value is True, the lap has passed as basic accuracy check for timing data. This does not guarantee accuracy but laps marked as inaccurate need to be handled with caution. They might contain errors which can not be spotted easily. Laps need to satisfy the following criteria to be marked as accurate:

not an inlap or outlap

set under green or yellow flag (the api sometimes has issues with data from SC/VSC laps)

is not the first lap after a safety car period (issues with SC/VSC might still appear on the first lap after it has ended)

has a value for lap time and all sector times

the sum of the sector times matches the lap time (If this were to ever occur, it would also be logged separately as a data integrity error. You usually don’t need to worry about this.)

Since in accurate lap time match the len of lap times with nans we will remove all laps not accurate from the database

In [46]:
qualifying_data_clean = qualifying_data_clean[qualifying_data_clean['qualifying_IsAccurate']]

In [47]:
qualifying_data_clean['qualifying_LapTime'].isna().sum()

0

Removing in accurate laps got rid of lap time nans

In [48]:
ds_ultils.nan_checker(qualifying_data_clean)

This dataframe has 83465 NaN values
 The NaN values come from: ['qualifying_SpeedI1', 'qualifying_SpeedI2', 'qualifying_SpeedFL', 'qualifying_SpeedST', 'qualifying_Compound', 'qualifying_TyreLife', 'qualifying_FreshTyre', 'qualifying_Stint', 'qualifying_DistanceToDriverAhead']. 



In [49]:
qualifying_data_clean.loc[qualifying_data_clean['qualifying_SpeedI1'].isna(),'qualifying_DriverNumber'].unique()

array([ 7., 33., 28., 44., 77.])

In [50]:
qualifying_data_clean.loc[qualifying_data_clean['qualifying_SpeedI1'].isna(),'qualifying_lapId'].unique()

array([ 33.,  41.,  98., 111.,   4.,  12.,  21.,  66.])

In [51]:
qualifying_data_clean.loc[qualifying_data_clean['qualifying_SpeedI1'].isna(),['qualifying_lapId','raceId','qualifying_DriverNumber','qualifying_SpeedI1']]

,qualifying_lapId,raceId,qualifying_DriverNumber,qualifying_SpeedI1
167010,33.0,1000,7.0,NaN
167011,33.0,1000,7.0,NaN
167012,33.0,1000,7.0,NaN
167013,33.0,1000,7.0,NaN
167014,33.0,1000,7.0,NaN
...,...,...,...,...
586811,66.0,1018,7.0,NaN
586812,66.0,1018,7.0,NaN
586813,66.0,1018,7.0,NaN
586814,66.0,1018,7.0,NaN


In [52]:
query = (qualifying_data_clean['qualifying_lapId']==33) & (qualifying_data_clean['raceId']==1000) & (qualifying_data_clean['qualifying_DriverNumber'] ==7)
print('percentage nans:',(qualifying_data_clean.loc[query,'qualifying_SpeedI1'].isna().sum()/len(qualifying_data_clean.loc[query,'qualifying_SpeedI1']))*100)
qualifying_data_clean.loc[query,['qualifying_SpeedI1','qualifying_Stint']]

percentage nans: 100.0


,qualifying_SpeedI1,qualifying_Stint
167010,NaN,2.0
167011,NaN,2.0
167012,NaN,2.0
167013,NaN,2.0
167014,NaN,2.0
...,...,...
167594,NaN,2.0
167595,NaN,2.0
167596,NaN,2.0
167597,NaN,2.0


In [53]:


qualifying_data_clean = fill_mean_parameter(qualifying_data_clean,'qualifying_SpeedI1')

Clean other parameters convert time to milliseconds save to csv

In [54]:
ds_ultils.nan_checker(qualifying_data_clean)

This dataframe has 78285 NaN values
 The NaN values come from: ['qualifying_SpeedI2', 'qualifying_SpeedFL', 'qualifying_SpeedST', 'qualifying_Compound', 'qualifying_TyreLife', 'qualifying_FreshTyre', 'qualifying_Stint', 'qualifying_DistanceToDriverAhead']. 



In [55]:
qualifying_data_clean = fill_mean_parameter(qualifying_data_clean,'qualifying_SpeedI2')
qualifying_data_clean = fill_mean_parameter(qualifying_data_clean,'qualifying_SpeedFL')
qualifying_data_clean = fill_mean_parameter(qualifying_data_clean,'qualifying_SpeedST')

In [56]:
qualifying_data_clean.loc[qualifying_data_clean['qualifying_Compound'].isna(),['qualifying_Compound', 'qualifying_TyreLife', 'qualifying_FreshTyre', 'qualifying_Stint']]

,qualifying_Compound,qualifying_TyreLife,qualifying_FreshTyre,qualifying_Stint
79477,NaN,NaN,NaN,NaN
79478,NaN,NaN,NaN,NaN
79479,NaN,NaN,NaN,NaN
79480,NaN,NaN,NaN,NaN
79481,NaN,NaN,NaN,NaN
...,...,...,...,...
546466,NaN,NaN,NaN,NaN
546467,NaN,NaN,NaN,NaN
546468,NaN,NaN,NaN,NaN
546469,NaN,NaN,NaN,NaN


In [57]:
print('compound nans :', qualifying_data_clean['qualifying_Compound'].isna().sum())
print('Tyrelife nans :',qualifying_data_clean['qualifying_TyreLife'].isna().sum())
print('FreshTyre nans :',qualifying_data_clean['qualifying_FreshTyre'].isna().sum())
print('Stint nans :',qualifying_data_clean['qualifying_Stint'].isna().sum())
print('distance to driver ahead nans :',qualifying_data_clean['qualifying_DistanceToDriverAhead'].isna().sum())


compound nans : 2470
Tyrelife nans : 2470
FreshTyre nans : 2470
Stint nans : 2470
distance to driver ahead nans : 59596


In [58]:
(qualifying_data_clean.loc[qualifying_data_clean['qualifying_Compound'].isna(),['qualifying_Compound', 'qualifying_TyreLife', 'qualifying_FreshTyre', 'qualifying_Stint']].isna().sum()/len(qualifying_data_clean))*100

qualifying_Compound     0.245075
qualifying_TyreLife     0.245075
qualifying_FreshTyre    0.245075
qualifying_Stint        0.245075
dtype: float64

Compount, tyreLife, freshtyre, stint all have there nans at the same rows. Drop these rows as 0.2% of the data

In [59]:
qualifying_data_clean=qualifying_data_clean[qualifying_data_clean.index.isin(qualifying_data_clean['qualifying_Compound'].dropna(axis=0).index)].copy()

In [60]:
(qualifying_data_clean['qualifying_DistanceToDriverAhead'].isna().sum()/len(qualifying_data_clean))*100

5.752826780613178

Distance to driver ahead every lap has 5% of the data missing. This can not be dropped and has to be filled. However for this lap it is always nan and even for the whole qualifying stint. Therefore since it is expected not to be a key parameter the missing values are filled with mean for that driver and race.

In [61]:
indexes = qualifying_data_clean.loc[qualifying_data_clean['qualifying_DistanceToDriverAhead'].isna()].groupby(['qualifying_lapId',
'raceId','qualifying_DriverNumber'])['qualifying_Stint'].unique().index
values = qualifying_data_clean.loc[qualifying_data_clean['qualifying_DistanceToDriverAhead'].isna()].groupby(['qualifying_lapId',
'raceId','qualifying_DriverNumber'])['qualifying_Stint'].unique().values
for i,ind in enumerate(indexes):   
    query = (qualifying_data_clean['raceId']==ind[1]) & (qualifying_data_clean['qualifying_DriverNumber'] ==ind[2])
    mean_para = qualifying_data_clean.loc[query,['qualifying_DistanceToDriverAhead']].mean()
    query2= (qualifying_data_clean['qualifying_lapId']==ind[0])&(qualifying_data_clean['raceId']==ind[1]) & (qualifying_data_clean['qualifying_DriverNumber'] ==ind[2])
    qualifying_data_clean.loc[query2,['qualifying_DistanceToDriverAhead']] = mean_para.values

In [62]:
ds_ultils.nan_checker(qualifying_data_clean)

This dataframe has 0 NaN values


The dataframe has no nulls and most of the duplicated information in the columns is removed

In [63]:
qualifying_data_clean.duplicated().sum()
# 0 duplicated rows

0

In [64]:
qualifying_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1005384 entries, 0 to 1018657
Data columns (total 77 columns):
 #   Column                            Non-Null Count    Dtype         
---  ------                            --------------    -----         
 0   raceId                            1005384 non-null  int64         
 1   year                              1005384 non-null  int64         
 2   name                              1005384 non-null  object        
 3   date                              1005384 non-null  datetime64[ns]
 4   time                              1005384 non-null  object        
 5   lat_x                             1005384 non-null  float64       
 6   lng_x                             1005384 non-null  float64       
 7   circuitRef                        1005384 non-null  object        
 8   country                           1005384 non-null  object        
 9   alt                               1005384 non-null  float64       
 10  qualifyId         

# Now need to check the time columns

In [65]:
qualifying_data_clean[['qualifying_LapTime','qualifying_Time_x','qualifying_Sector1Time',
       'qualifying_Sector2Time', 'qualifying_Sector3Time','qualifying_Sector1SessionTime', 'qualifying_Sector2SessionTime',
       'qualifying_Sector3SessionTime','qualifying_LapStartTime','qualifying_LapStartDate','qualifying_Date',
       'qualifying_SessionTime', 'qualifying_Time_y']]

,qualifying_LapTime,qualifying_Time_x,qualifying_Sector1Time,qualifying_Sector2Time,qualifying_Sector3Time,qualifying_Sector1SessionTime,qualifying_Sector2SessionTime,qualifying_Sector3SessionTime,qualifying_LapStartTime,qualifying_LapStartDate,qualifying_Date,qualifying_SessionTime,qualifying_Time_y
0,0 days 00:01:04.539000,0 days 00:08:43.423000,0 days 00:00:16.343000,0 days 00:00:28.620000,0 days 00:00:19.576000,0 days 00:07:55.227000,0 days 00:08:23.847000,0 days 00:08:43.423000,0 days 00:07:38.884000,2018-06-30 13:03:47.133,2018-06-30 13:03:47.133,0 days 00:07:38.884000,0 days 00:00:00
1,0 days 00:01:04.539000,0 days 00:08:43.423000,0 days 00:00:16.343000,0 days 00:00:28.620000,0 days 00:00:19.576000,0 days 00:07:55.227000,0 days 00:08:23.847000,0 days 00:08:43.423000,0 days 00:07:38.884000,2018-06-30 13:03:47.133,2018-06-30 13:03:47.280,0 days 00:07:39.031000,0 days 00:00:00.147000
2,0 days 00:01:04.539000,0 days 00:08:43.423000,0 days 00:00:16.343000,0 days 00:00:28.620000,0 days 00:00:19.576000,0 days 00:07:55.227000,0 days 00:08:23.847000,0 days 00:08:43.423000,0 days 00:07:38.884000,2018-06-30 13:03:47.133,2018-06-30 13:03:47.324,0 days 00:07:39.075000,0 days 00:00:00.191000
3,0 days 00:01:04.539000,0 days 00:08:43.423000,0 days 00:00:16.343000,0 days 00:00:28.620000,0 days 00:00:19.576000,0 days 00:07:55.227000,0 days 00:08:23.847000,0 days 00:08:43.423000,0 days 00:07:38.884000,2018-06-30 13:03:47.133,2018-06-30 13:03:47.564,0 days 00:07:39.315000,0 days 00:00:00.431000
4,0 days 00:01:04.539000,0 days 00:08:43.423000,0 days 00:00:16.343000,0 days 00:00:28.620000,0 days 00:00:19.576000,0 days 00:07:55.227000,0 days 00:08:23.847000,0 days 00:08:43.423000,0 days 00:07:38.884000,2018-06-30 13:03:47.133,2018-06-30 13:03:47.579,0 days 00:07:39.330000,0 days 00:00:00.446000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018653,0 days 00:01:20.547000,0 days 00:32:57.064000,0 days 00:00:22.227000,0 days 00:00:30.223000,0 days 00:00:28.097000,0 days 00:31:58.744000,0 days 00:32:28.967000,0 days 00:32:57.064000,0 days 00:31:36.517000,2019-05-11 13:16:36.564,2019-05-11 13:17:56.609,0 days 00:32:56.562000,0 days 00:01:20.045000
1018654,0 days 00:01:20.547000,0 days 00:32:57.064000,0 days 00:00:22.227000,0 days 00:00:30.223000,0 days 00:00:28.097000,0 days 00:31:58.744000,0 days 00:32:28.967000,0 days 00:32:57.064000,0 days 00:31:36.517000,2019-05-11 13:16:36.564,2019-05-11 13:17:56.810,0 days 00:32:56.763000,0 days 00:01:20.246000
1018655,0 days 00:01:20.547000,0 days 00:32:57.064000,0 days 00:00:22.227000,0 days 00:00:30.223000,0 days 00:00:28.097000,0 days 00:31:58.744000,0 days 00:32:28.967000,0 days 00:32:57.064000,0 days 00:31:36.517000,2019-05-11 13:16:36.564,2019-05-11 13:17:56.849,0 days 00:32:56.802000,0 days 00:01:20.285000
1018656,0 days 00:01:20.547000,0 days 00:32:57.064000,0 days 00:00:22.227000,0 days 00:00:30.223000,0 days 00:00:28.097000,0 days 00:31:58.744000,0 days 00:32:28.967000,0 days 00:32:57.064000,0 days 00:31:36.517000,2019-05-11 13:16:36.564,2019-05-11 13:17:57.050,0 days 00:32:57.003000,0 days 00:01:20.486000


- `Time_x` (pandas.Timedelta): Session time when the lap time was set (end of lap) 
- `LapTime` (pandas.Timedelta): Recorded lap time. 
- `Sector1Time` (pandas.Timedelta): Sector 1 recorded time
- `Sector2Time` (pandas.Timedelta): Sector 2 recorded time
- `Sector3Time` (pandas.Timedelta): Sector 3 recorded time
- `Sector1SessionTime` (pandas.Timedelta): Session time when the Sector 1 time was set
- `Sector2SessionTime` (pandas.Timedelta): Session time when the Sector 2 time was set
- `Sector3SessionTime` (pandas.Timedelta): Session time when the Sector 3 time was set
- `LapStartTime` (pandas.Timedelta): Session time at the start of the lap
- `LapStartDate` (pandas.Timestamp): Timestamp at the start of the lap
- `Time_y` (timedelta): Time (0 is start of the data slice)
- `SessionTime` (timedelta): Time elapsed since the start of the session
- `Date` (datetime): The full date + time at which this sample was created


`Laptime`, `sector` times are useful performance metrics

When the laptime occured in the session could be useful for when the track is increasing in performance

The times of the session when the sectors where recorded can be recreated from session time and lap time, and sectors time

`LapStartTime` & `LapStartDate` repeat each other therefore we will keep only `LapStartDate`

`Time_y` is the time delta from 0 at start of the lap, very useful for metrics around the lap

`SessionTime` and `Date` around the lap are not needed as they can be recreated if need from other metrics

In [66]:
qualifying_data_clean.drop(columns=['qualifying_Sector1SessionTime',
'qualifying_Sector2SessionTime',
'qualifying_Sector3SessionTime',
'qualifying_LapStartTime',
'qualifying_Date',
'qualifying_SessionTime',    
],axis=1, inplace=True)

In [67]:
qualifying_data_clean.rename(columns ={'qualifying_Time_x':'qualifying_end_lap_sessiontime','qualifying_Time_y':'qualifying_lap_timedelta'},inplace=True)

In [68]:
qualifying_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1005384 entries, 0 to 1018657
Data columns (total 71 columns):
 #   Column                            Non-Null Count    Dtype         
---  ------                            --------------    -----         
 0   raceId                            1005384 non-null  int64         
 1   year                              1005384 non-null  int64         
 2   name                              1005384 non-null  object        
 3   date                              1005384 non-null  datetime64[ns]
 4   time                              1005384 non-null  object        
 5   lat_x                             1005384 non-null  float64       
 6   lng_x                             1005384 non-null  float64       
 7   circuitRef                        1005384 non-null  object        
 8   country                           1005384 non-null  object        
 9   alt                               1005384 non-null  float64       
 10  qualifyId         

In [69]:
qualifying_data_clean['qualifying_end_lap_sessiontime'] = qualifying_data_clean['qualifying_end_lap_sessiontime'].astype('timedelta64')

In [70]:
qualifying_data_clean['qualifying_LapTime'] = qualifying_data_clean['qualifying_LapTime'].astype('timedelta64')
qualifying_data_clean['qualifying_Sector1Time'] = qualifying_data_clean['qualifying_Sector1Time'].astype('timedelta64')
qualifying_data_clean['qualifying_Sector2Time'] = qualifying_data_clean['qualifying_Sector2Time'].astype('timedelta64')
qualifying_data_clean['qualifying_Sector3Time'] = qualifying_data_clean['qualifying_Sector3Time'].astype('timedelta64')
qualifying_data_clean['qualifying_LapStartDate'] = qualifying_data_clean['qualifying_LapStartDate'].astype('datetime64')
qualifying_data_clean['qualifying_lap_timedelta'] = qualifying_data_clean['qualifying_lap_timedelta'].astype('timedelta64')

In [71]:
qualifying_data_clean['qualifying_LapTime'].dtype == '<m8[ns]'

True

In [72]:
# convert timedeltas to total seconds
column_data = list(qualifying_data_clean.columns)
for col in column_data:
    if qualifying_data_clean[col].dtype == '<m8[ns]':
        print(col)
        qualifying_data_clean[col] = qualifying_data_clean[col].dt.total_seconds()

qualifying_end_lap_sessiontime
qualifying_LapTime
qualifying_Sector1Time
qualifying_Sector2Time
qualifying_Sector3Time
qualifying_lap_timedelta


In [73]:
qualifying_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1005384 entries, 0 to 1018657
Data columns (total 71 columns):
 #   Column                            Non-Null Count    Dtype         
---  ------                            --------------    -----         
 0   raceId                            1005384 non-null  int64         
 1   year                              1005384 non-null  int64         
 2   name                              1005384 non-null  object        
 3   date                              1005384 non-null  datetime64[ns]
 4   time                              1005384 non-null  object        
 5   lat_x                             1005384 non-null  float64       
 6   lng_x                             1005384 non-null  float64       
 7   circuitRef                        1005384 non-null  object        
 8   country                           1005384 non-null  object        
 9   alt                               1005384 non-null  float64       
 10  qualifyId         

In [74]:
qualifying_data_clean.loc[(qualifying_data_clean['qualifying_Team'].str.lower()!=qualifying_data_clean['constructorRef']),'qualifying_Team'].unique()

array(['Red Bull Racing', 'Haas F1 Team', 'Force India', 'Toro Rosso',
       'Racing Point', 'Alfa Romeo Racing'], dtype=object)

In [75]:
qualifying_data_clean.loc[(qualifying_data_clean['qualifying_Team'].str.lower()!=qualifying_data_clean['constructorRef']),'constructorRef'].unique()

array(['red_bull', 'haas', 'force_india', 'toro_rosso', 'alfa',
       'racing_point'], dtype=object)

constructorRef does match qualifying_team it just has slightly different spelling. we will drop qualifying_team as the constructorRef comes from the larger Ergast database

In [76]:
qualifying_data_clean.loc[(qualifying_data_clean['qualifying_Driver'].str.lower()!=qualifying_data_clean['driverRef']),'qualifying_Driver'].unique()

array(['BOT', 'HAM', 'VET', 'RAI', 'VER', 'GRO', 'RIC', 'MAG', 'SAI',
       'HUL', 'OCO', 'GAS', 'LEC', 'ALO', 'STR', 'VAN', 'PER', 'SIR',
       'HAR', 'ERI', 'NOR', 'GIO', 'ALB', 'KVY', 'RUS', 'KUB'],
      dtype=object)

qualifyingRef uses the 3 letter acroynm instead of driverRef string, this is a repeat of the same information so will drop qualifying_Driver

In [77]:
qualifying_data_clean.drop(columns=['qualifying_Team','qualifying_Driver'],axis=1,inplace=True)

In [85]:
qualifying_data_clean.to_pickle('./data/clean/years_18_19_telemetry_subsetcircuits_cleaned.pkl',compression='gzip')
